In [6]:
import pandas as pd
import numpy as np
import pandas_gbq as gbq
import pydata_google_auth
import seaborn as sns
import matplotlib.pylab as plt
import datetime
import sys
sys.path.append('/home3/r05322021/Desktop/FB_hatecrime/Code/python/')
import fbscore

In [7]:
def bigquery_auth():
    SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',]
    
    credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,)

bigquery_auth()

In [8]:
query = '''
WITH OLD_USER AS(
SELECT user_id
FROM `ntufbdata.user_type.user_entering_type`
WHERE TYPE = 'WHOLE'
), REACTION AS(
(SELECT user_id,
        SPLIT(post_id, '_')[ORDINAL(1)] AS page_id,
        post_id
FROM `ntufbdata.USdata.1000_page_us_user_like_post_201501_to_201611_all`
WHERE TIMESTAMP(post_created_date_CT) >= TIMESTAMP('2015-05-03')
AND TIMESTAMP(post_created_date_CT) < TIMESTAMP('2015-05-31'))
UNION DISTINCT
(SELECT user_id,
        SPLIT(post_id, '_')[ORDINAL(1)] AS page_id,
        post_id
FROM `ntufbdata.USdata.politician_us_user_post_like_all`
WHERE TIMESTAMP(post_created_date_CT) >= TIMESTAMP('2015-05-03')
AND TIMESTAMP(post_created_date_CT) < TIMESTAMP('2015-05-31'))
)

SELECT user_id,
        STRING_AGG(page_id, ',') AS like_pages,
        STRING_AGG(CAST(like_time AS STRING), ',') AS like_times
FROM(
SELECT OLD_USER.user_id,
        REACTION.page_id,
        COUNT(*) AS like_time
FROM OLD_USER
INNER JOIN REACTION ON OLD_USER.user_id = REACTION.user_id
GROUP BY OLD_USER.user_id, REACTION.page_id)
GROUP BY user_id
'''

In [ ]:
user_type = gbq.read_gbq(type_query, project_id='ntufbdata')